# DTN Transfers over NVMEoF

This notebook demonstrates a copy from one DTN to another DTN using NVME over fabric (NVMEoF).
Current limitations:
- Only single NMVE drives are supported at this time - mdraid support is in progress
- For each NVME drive, only one partition is supported at this time - if there are multiple partitions only the first will be used

In [ ]:
# initialize client and run a connectivity check to the orchestrator container
from dtn_orchestrator.client import DTNOrchestratorClient
client = DTNOrchestratorClient()
client.check()

In [ ]:
# register DTNs with their management address - this will reuse existing objects if they're already registered
# sender DTN
dtn1 = client.register_dtn('<ip1>:5000')
# receiver DTN
dtn2 = client.register_dtn('<ip2>:5000')

In [ ]:
# Run latency between DTNs to test connectivity
print(dtn1.ping(dtn2.id))
print(dtn2.ping(dtn1.id))

In [ ]:
# Set up NVMEoF connection between DTNs - dtn1's drive will be accessed/mounted on dtn2
conn = client.setup_nvmeof(dtn1, dtn2)
# At this point DTN1's drive shows up on DTN2 - use 'nvme list' in dtn2's agent container to verify

In [ ]:
# Get a list of files on each DTN
print([f['name'] for f in dtn1.files() if f['type'] == 'file'])
print([f['name'] for f in dtn2.files() if f['type'] == 'file'])

In [ ]:
# Copy the first file found on DTN1 to the first (hopefully empty) file on DTN2
# This is done with dd on dtn2 only
result = conn.copy([dtn1.files()[0]['name']], [dtn2.files()[0]['name']])
print(result.get('transfer'))

In [ ]:
# Get transfer status from the orchestrator
client.get_transfer(result['transfer'])

In [ ]:
# Unmount remote drive and shutdown the NVMEoF connection
conn.disconnect()
# note: for future connections to work dtn1's NVME ports may need to be cleared manually.
# If this is the case run 'nvmetcli clear' to reset